In [ ]:
import matplotlib.pyplot as plt
import numdifftools as nd
import progressbar
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import pickle

def sign_val(a):
    if(a>1e-4):
        return 1
    elif(a<-1e-4):
        return -1
    else:
        return 0
def coth(x):
    return 1 / math.tanh(x)
plank_constant = 1
tau = 30
omega_1 = 0.1
omega_2 = 0.5
beta_1 = 1.0
beta_2 = 0.75
adam_beta_1 = 0.9
adam_beta_2 = 0.999
adam_learning_rate = 0.01
adam_epsilon = 1e-8
constant_part = coth(beta_1*plank_constant*omega_1)*plank_constant*math.sqrt(3)/(4*tau)

def reLU(val):
  return np.maximum(0, val)
def reLU_derivative(z):
    if(z>0):
        return 1
    if(z<0):
        return 0
    if(z==0):
        print("z = 0 , NN is giving same value at both zero and tau ")

Q4 = plank_constant*omega_1*0.5*(coth((beta_1*omega_1*plank_constant*0.5))-coth((beta_2*omega_2*plank_constant*0.5)))
W1 = plank_constant*omega_2*0.5*(1-(omega_1/omega_2))*coth(beta_1*omega_1*plank_constant*0.5)
W2 = plank_constant*omega_1*0.5*(1-(omega_2/omega_1))*coth(beta_2*omega_2*plank_constant*0.5)

# figure of merit vs tau

def figure_of_merit(pt_tau):
    return Q4**2/((W1+W2+net.C_AB(pt_tau))*2*pt_tau)
def cooling_efficiency(pt_tau):
    return Q4/(W1+W2+net.C_AB(pt_tau))
def polynomial_ansatz( t ):
  w1 = 0.1
  w2 = 0.5
  a3 = 10.0
  a4 = -15.0
  a5 = 6.0
  tau = 6
  return w1 + (w2 - w1)*( a3*((t/tau)**3) + a4*((t/tau)**4) + a5*((t/tau)**5) )

def polynomial_ansatz_first( t ):
  w1 = 0.1
  w2 = 0.5
  a3 = 10.0
  a4 = -15.0
  a5 = 6.0
  tau = 6
  return   ((w2 - w1)/tau)*( 3*a3*((t/tau)**2) + 4*a4*((t/tau)**3) + 5*a5*((t/tau)**4) )
def polynomial_ansatz_second( t ):
  w1 = 0.1
  w2 = 0.5
  a3 = 10.0
  a4 = -15.0
  a5 = 6.0
  tau = 6
  return ((w2 - w1)/tau**2)*( 6*a3*((t/tau)) + 12*a4*((t/tau)**2) + 20*a5*((t/tau)**3) )


In [ ]:

first_derivative  = []
second_derivative  = []
# use net class from the main script to call the net.gradients_wrt_time method
for t in time:
    output_time0,grad_output_time0,grad_grad_output_time0 = net.gradients_wrt_time(t)
    first_derivative.append(grad_output_time0.item())
    second_derivative.append(grad_grad_output_time0.item())
plt.plot(time , first_derivative)
plt.plot(time , second_derivative)
# plt.plot(t_val,dw_val)
# plt.plot(t_val,ddw_val)

In [ ]:
t_val = np.arange(0,6,0.01)
w_val_first = []
w_val_second = []
for t in t_val:
  w_val_first.append(polynomial_ansatz_first(t))
  w_val_second.append(polynomial_ansatz_second(t))

w_val_first = np.array(w_val_first)
w_val_second = np.array(w_val_second)


In [ ]:
t_val = np.arange(0,6,0.01)
w_val = []
for t in t_val:
  w_val.append(polynomial_ansatz( t ))
w_val = np.array(w_val)

In [ ]:
# df_polynomial_ansatz = nd.Derivative(polynomial_ansatz, n=1)
# dw_val = df_polynomial_ansatz(t_val)
# ddf_polynomial_ansatz = nd.Derivative(polynomial_ansatz, n=2)
# ddw_val = ddf_polynomial_ansatz(t_val)
# # t_val_normalized = []
# for i in t_val:
#   t_val_normalized.append(i/2)
# t_val_normalized = np.array(t_val_normalized)
import numpy as np
import matplotlib.pyplot as plt
# with open('para_268_min_1000stepsX2.pkl', 'wb') as f:file_name = "para_best_yet.pkl"
import pickle
file_name = "para_tau_30_extra_trainoing_with_term_1_to_7.pkl"
# Open the file in binary read mode
with open(file_name, "rb") as file:
    param_step = pickle.load(file)    
i = 0
for param in net.parameters():
            
    # if(a==0):
    #     a = 1
    # print(param.data)

    param.data = param_step[0][i]
    i =  i + 1


# Open the file in binary read mode
# with open(file_name, "rb") as file:
#     param_step = pickle.load(file)    
omega = []
time = np.arange(0, 6, 0.01)

time = np.arange(0, 6, 0.01)
sigma = []
# grad = []
for t in time:
    grad  =  net.gradients_wrt_time(t)
    sigma.append(np.sqrt( ((grad[0].item())**2) - 0.75*(((grad[1].item())**2)/((grad[0].item())**2)) + 0.5*((grad[2].item())/(grad[0].item())) ))
for t in time:
    input_data = torch.tensor([[1,t,t**2,t**3,t**4,t**5,t**6,t**7,t**8,t**9]], dtype=torch.float32, requires_grad=True)
    omega.append(net.forward(input_data).item())


In [ ]:
plt.figure(figsize=(15,7.5))
plt.plot(t_val,w_val)
plt.plot(time,omega)
